# Air Boltic Analytics
***

## Scope
- Business context
- Exploratory data analysis
- Data modeling
    - Conceptual model
    - Logical model
- Analytics engineering
    - Ingestion
    - Transformation
    - Serving (out of scope)
    - DataOps
        - CI/CD pipeline
- Evolution
***

## Business context
> *Air Boltic is a marketplace for sharing aeroplane rides. On one side of the market there are
aeroplane operators, ready to fly from anywhere to anywhere. On the other side of the
market there are individuals and groups who need to get from A to B.*
> 
> *After a few months of operating in Europe, we see that the service is doing great in some
regions, while it hasn’t taken off in others areas. We want to replicate the success in all of
our regions and expand outwards, envisioning to facilitate 20% of all aeroplane rides across
the globe in 2030. To achieve the goal, we need to understand what are the drivers of
growth: the kind of customers we serve well, the kind of use cases we cover (short or long,
from where to where, low cost or premium, small or big planes etc). To make sure the
business fits into our portfolio, we also monitor daily/weekly/monthly active users, revenue
and other metrics that we can compare to our older business lines.*

We have information about a hypothetic aeroplane ride sharing marketplace:
- Customers
- Customer groups
- Aeroplanes
- Trips
- Orders
- Aeroplane models

The business stakeholders need:
 - **Data model** that enables monitoring and self-service analysis of the service
 - Design of **CI/CD process**

Based on the description, the service operates using with following high-level entities:

```mermaid

mindmap
  root((Order))
    Status
    Trip
        Aeroplane
            Aeroplane model
    Customer
        Customer group
```

## Exploratory data analysis
As the first step, let us perform quick exploratory data analysis (EDA) to get general understanding of the data available,
 its quality, and basic statistics.

The sample data is provided as a collection of CSV and JSON files (already on the local filesystem). 

We are dealing with the following types of data:
- Structured data
- Semistructured data

In [1]:
import json
import polars as pl
import duckdb


SAMPLES_DIR = '../samples'
CSV_FILENAMES = ['customer', 'customer_group', 'aeroplane', 'order', 'trip']
DEV_DB = 'dev.duckdb'

### EDA: Customer
We can preview the dataset in the command line using [DuckDB](https://duckdb.org/):

In [2]:
!head -n 10 ../samples/customer.csv | duckdb -c "SELECT * FROM read_csv('/dev/stdin') LIMIT 3;"

┌─────────────┬───────────────┬───────────────────┬──────────────────────────────┬──────────────────┐
│ Customer ID │     Name      │ Customer Group ID │            Email             │   Phone Number   │
│    int64    │    varchar    │       int64       │           varchar            │     varchar      │
├─────────────┼───────────────┼───────────────────┼──────────────────────────────┼──────────────────┤
│           1 │ John Doe      │                 1 │ john.doe@gmail.com           │ +1-555-123-4567  │
│           2 │ Jane Smith    │              NULL │ jane.smith@yahoo.com         │ +44 20 7946 0958 │
│           3 │ Alice Johnson │                 3 │ alice.j@example.com          │ +49-30-12345678  │
└─────────────┴───────────────┴───────────────────┴──────────────────────────────┴──────────────────┘


However, it's better to use a specialized Python package - for example, [Polars](https://pola.rs/).

In [3]:
df_customer = pl.read_csv(f'{SAMPLES_DIR}/customer.csv')
df_customer.head()

Customer ID,Name,Customer Group ID,Email,Phone Number
i64,str,i64,str,str
1,"""John Doe""",1,"""john.doe@gmail.com ""","""+1-555-123-4567"""
2,"""Jane Smith""",null,"""jane.smith@yahoo.com ""","""+44 20 7946 0958"""
3,"""Alice Johnson""",3,"""alice.j@example.com ""","""+49-30-12345678"""
4,"""Bob Brown""",4,"""bob.brown@outlook.com ""","""+61-2-9876-5432"""
5,"""Carol White""",null,"""carol.white@company.com ""","""+1 (555) 567-8901"""


In [4]:
df_customer.describe()

statistic,Customer ID,Name,Customer Group ID,Email,Phone Number
str,f64,str,f64,str,str
"""count""",20.0,"""20""",14.0,"""18""","""17"""
"""null_count""",0.0,"""0""",6.0,"""2""","""3"""
"""mean""",10.5,null,5.428571,null,null
"""std""",5.91608,null,3.10618,null,null
"""min""",1.0,"""Alice Johnson""",1.0,"""alice.j@example.com ""","""+1 (555) 012-4567"""
"""25%""",6.0,null,3.0,null,null
"""50%""",11.0,null,6.0,null,null
"""75%""",15.0,null,8.0,null,null
"""max""",20.0,"""Rachel Walker""",10.0,"""rachel.walker@company.net ""","""+91-22-1234-5678"""


Our observations:
- Primary key - `Customer ID`
- There are missing values in `Customer Group ID`, `Email`, and `Phone Number`
- Phone number format is not consistent across the dataset

### EDA: Customer group

In [5]:
df_customer_group = pl.read_csv(f'{SAMPLES_DIR}/customer_group.csv')
df_customer_group.head()

ID,Type,Name,Registry number
i64,str,str,str
1,"""Company""","""Bolt""","""5421524153"""
2,"""Company""","""BigBang""","""131101002"""
3,"""Company""","""Lofty""","""1533153"""
4,"""Private Group""","""Mari's friends""",null
5,"""Organisation""","""Non-profit for dog safety""","""AT452453"""


In [6]:
df_customer_group.describe()

statistic,ID,Type,Name,Registry number
str,f64,str,str,str
"""count""",5.0,"""5""","""5""","""4"""
"""null_count""",0.0,"""0""","""0""","""1"""
"""mean""",3.0,null,null,null
"""std""",1.581139,null,null,null
"""min""",1.0,"""Company""","""BigBang""","""131101002"""
"""25%""",2.0,null,null,null
"""50%""",3.0,null,null,null
"""75%""",4.0,null,null,null
"""max""",5.0,"""Private Group""","""Non-profit for dog safety""","""AT452453"""


Our observations:
- Primary key - `ID`
- Missing values and non-consistent format in `Registry number`

### EDA: Aeroplane

In [7]:
df_aeroplane = pl.read_csv(f'{SAMPLES_DIR}/aeroplane.csv')
df_aeroplane.head()

Airplane ID,Airplane Model,Manufacturer
i64,str,str
1,"""737-800""","""Boeing"""
2,"""A320neo""","""Airbus"""
3,"""E190-E2""","""Embraer"""
4,"""CRJ900""","""Bombardier"""
5,"""737-800""","""Boeing"""


In [8]:
df_aeroplane.describe()

statistic,Airplane ID,Airplane Model,Manufacturer
str,f64,str,str
"""count""",10.0,"""10""","""10"""
"""null_count""",0.0,"""0""","""0"""
"""mean""",5.5,null,null
"""std""",3.02765,null,null
"""min""",1.0,"""737-800""","""Airbus"""
"""25%""",3.0,null,null
"""50%""",6.0,null,null
"""75%""",8.0,null,null
"""max""",10.0,"""G650""","""Gulfstream"""


Our observations:
- Primary key - `Airplane ID`
- No modeling-related DQ issues

### EDA: Trip

In [9]:
df_trip = pl.read_csv(f'{SAMPLES_DIR}/trip.csv')
df_trip.head()

Trip ID,Origin City,Destination City,Airplane ID,Start Timestamp,End Timestamp
i64,str,str,i64,str,str
1,"""New York""","""London""",1,"""2024-08-01 14:30:00""","""2024-08-02 02:00:00"""
2,"""Tokyo""","""Paris""",2,"""2024-08-03 09:00:00""","""2024-08-03 17:00:00"""
3,"""Sydney""","""Los Angeles""",3,"""2024-08-05 22:00:00""","""2024-08-05 15:00:00"""
4,"""Dubai""","""Singapore""",4,"""2024-08-07 06:00:00""","""2024-08-07 10:30:00"""
5,"""Berlin""","""Madrid""",5,"""2024-08-09 16:00:00""","""2024-08-09 19:00:00"""


In [10]:
df_trip.describe()

statistic,Trip ID,Origin City,Destination City,Airplane ID,Start Timestamp,End Timestamp
str,f64,str,str,f64,str,str
"""count""",20.0,"""20""","""20""",20.0,"""20""","""20"""
"""null_count""",0.0,"""0""","""0""",0.0,"""0""","""0"""
"""mean""",10.5,null,null,5.5,null,null
"""std""",5.91608,null,null,2.946898,null,null
"""min""",1.0,"""Amsterdam""","""Auckland""",1.0,"""2024-08-01 14:30:00""","""2024-08-02 02:00:00"""
"""25%""",6.0,null,null,3.0,null,null
"""50%""",11.0,null,null,6.0,null,null
"""75%""",15.0,null,null,8.0,null,null
"""max""",20.0,"""Vancouver""","""Tokyo""",10.0,"""2024-08-28 09:00:00""","""2024-08-28 13:30:00"""


Our observations:
- Primary key - `Trip ID`
- No modeling-related DQ issues

### EDA: Order

In [11]:
df_order = pl.read_csv(f'{SAMPLES_DIR}/order.csv')
df_order.head()

Order ID,Customer ID,Trip ID,Price (EUR),Seat No,Status
i64,i64,i64,i64,str,str
1,3,10,1200,"""12A""","""Finished"""
2,5,7,1800,"""8B""","""Finished"""
3,8,2,1500,"""15C""","""Finished"""
4,12,8,1300,"""21D""","""Finished"""
5,16,8,700,"""3E""","""Finished"""


In [12]:
df_order.describe()

statistic,Order ID,Customer ID,Trip ID,Price (EUR),Seat No,Status
str,f64,f64,f64,f64,str,str
"""count""",20.0,20.0,20.0,20.0,"""20""","""20"""
"""null_count""",0.0,0.0,0.0,0.0,"""0""","""0"""
"""mean""",10.5,10.5,5.95,1510.0,null,null
"""std""",5.91608,5.91608,2.605157,606.890261,null,null
"""min""",1.0,1.0,2.0,500.0,"""10E""","""Booked"""
"""25%""",6.0,6.0,4.0,1100.0,null,null
"""50%""",11.0,11.0,6.0,1600.0,null,null
"""75%""",15.0,15.0,8.0,2000.0,null,null
"""max""",20.0,20.0,10.0,2500.0,"""9F""","""Finished"""


Our observations:
- Primary key - `Order ID`
- No modeling-related DQ issues

### Aeroplane model

In [13]:
df_aeroplane_model = pl.read_json(f'{SAMPLES_DIR}/aeroplane_model.json')
df_aeroplane_model.head()

Boeing,Airbus,Embraer,Bombardier,Cessna,Gulfstream,Dassault,Mitsubishi,COMAC
struct[3],struct[3],struct[3],struct[2],struct[2],struct[2],struct[2],struct[1],struct[1]
"{{189,79015,2935,""CFM56-7B""},{396,351535,7930,""GE90-115B""},{296,254000,7635,""GEnx-1B""}}","{{194,79000,3700,""CFM LEAP-1A""},{440,280000,8100,""Rolls-Royce Trent XWB""},{853,560000,8000,""Rolls-Royce Trent 900""}}","{{114,56600,3250,""PW1900G""},{88,37500,2200,""CF34-8E""},{146,61700,2600,""PW1900G""}}","{{90,38530,1554,""CF34-8C5""},{90,29257,1567,""PW150A""}}","{{12,16555,3516,""AE3007C2""},{9,13605,2700,""PW306D1""}}","{{19,45360,7500,""BR725""},{19,41300,6750,""BR710""}}","{{19,31750,5950,""PW307A""},{10,19105,4000,""PW308C""}}","{{92,39000,2030,""PW1200G""}}","{{174,72500,2555,""CFM LEAP-1C""}}"


In [14]:
df_aeroplane_model.describe()

statistic,Boeing,Airbus,Embraer,Bombardier,Cessna,Gulfstream,Dassault,Mitsubishi,COMAC
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null
"""min""",null,null,null,null,null,null,null,null,null
"""25%""",null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null
"""max""",null,null,null,null,null,null,null,null,null


Our observations:
- Semi-structured dataset with unique top-level key representing manufacturer
- No modeling-related DQ issues

***

## Data Modeling

We are done with understanding the business context and EDA, which are essential steps before starting any modeling or engineering activity.
 
Now we can do data modeling. 

For diagrams I will use a code-first tool called [Mermaid](https://mermaid.js.org/).

### Conceptual model

```mermaid

graph TB;  
    order -- placed by --> customer;
    customer -- belongs to --> customer_group;
    order -- is seat reservation for --> trip;
    trip -- is carried out using --> aeroplane;
    aeroplane -- is instance of --> aeroplane_model;
```

### Logical model

As of today, the most commonly adopted approach to modeling data for analytics is **dimensional modeling** originally developed by Ralph Kimball. I will stick to this approach and implement Star Schema.

```mermaid
erDiagram
    fct_order {
        integer order_id PK
        integer customer_id FK
        integer trip_id FK
        float price
        string seat_no
        string status
    }
    dim_customer {
        integer customer_id PK
        string customer_name
        integer customer_group_id FK
        string email
    }
    dim_customer_group {
        integer customer_group_id PK
        string customer_group_type
        string customer_group_name
        string registry_number
    }
    dim_aeroplane {
        integer airplane_id PK
        string airplane_model FK
        string manufacturer FK
    }
    dim_aeroplane_model {
        string manufacturer PK
        string airplane_model PK
        integer max_seats
        integer max_weight
        integer max_distance
        string engine_type
    }
    dim_trip {
        integer trip_id PK
        string origin_city
        string destination_city
        integer airplane_id FK
        timestamp start_timestamp
        timestamp end_timestamp
    }

    dim_customer_group ||--o{ dim_customer : customer_group_id
    dim_aeroplane_model ||--o{ dim_aeroplane : "manufacturer, airplane_model"
    dim_aeroplane ||--o{ dim_trip : airplane_id
    dim_customer ||--o{ fct_order : customer_id
    dim_trip ||--o{ fct_order : trip_id

```

There are a few considerations here. 

**Note 1:** It is a *logical model*, its primary purpose it to reflect the entities, attributes, and relationships. Depending on the target DWH technology, the *physical model* can be represented by a different set of tables, database-specific data types, and may not contain Foreign Keys at all. Also, analytical models tend to be less normalized, especially when dealing with large datasets in distributed environments. In this project, the resulting physical model (star schema) will include only `fct_order`, `dim_trip`, `dim_aeroplane`, and `dim_customer`.

**Note 2:** Graphs are getting more and more popular, but are out of scope for this project. 

**Note 3:** There are other approaches in the industry (e.g. OBT, ensemble modeling). I will be happy to discuss these as well :)

## Ingestion

### Aeroplane model

In [15]:
# Read into a Polars df

with open(f'{SAMPLES_DIR}/aeroplane_model.json', 'r') as f:
    json_data = json.load(f)

aeroplane_models_lst = []

for manufacturer, models in json_data.items():
    for model, attrs in models.items():
        model_record = {
            'manufacturer': manufacturer,
            'model': model,
            **attrs
        }
        aeroplane_models_lst.append(model_record)

df_aeroplane_model = pl.DataFrame(aeroplane_models_lst)
df_aeroplane_model.sample(n=3)

manufacturer,model,max_seats,max_weight,max_distance,engine_type
str,str,i64,i64,i64,str
"""Dassault""","""Falcon 7X""",19,31750,5950,"""PW307A"""
"""Boeing""","""777-300ER""",396,351535,7930,"""GE90-115B"""
"""Bombardier""","""Q400""",90,29257,1567,"""PW150A"""


In [16]:
# Ingest into DuckDB
# I will load data into schema `staging` (could be `bronze` in Medallion Architecture by Databricks)

arrow_aeroplane_model = df_aeroplane_model.to_arrow()

with duckdb.connect(DEV_DB) as db:
    db.execute('CREATE SCHEMA IF NOT EXISTS staging')
    db.register('arrow_aeroplane_model', arrow_aeroplane_model)
    db.execute('CREATE TABLE IF NOT EXISTS staging.raw_aeroplane_model AS SELECT * FROM arrow_aeroplane_model')

    stg_raw_aeroplane_model_sample = db.execute('SELECT * FROM staging.raw_aeroplane_model LIMIT 3').fetchdf()
    print(stg_raw_aeroplane_model_sample)

  manufacturer      model  max_seats  max_weight  max_distance engine_type
0       Boeing    737-800        189       79015          2935    CFM56-7B
1       Boeing  777-300ER        396      351535          7930   GE90-115B
2       Boeing      787-9        296      254000          7635     GEnx-1B


### Customer, Customer group, Aeroplane, Order, Trip

In [17]:
print(*CSV_FILENAMES, sep='\n')

customer
customer_group
aeroplane
order
trip


In [18]:
# Since we have several files of the same type, makes sense to wrap the ingestion code into a function :)

def load_csv(filename, preview_sample_size):
    df = pl.read_csv(f'{SAMPLES_DIR}/{filename}.csv')
    
    print(f"***\nLoading {filename} data... \nSample:\n{df.sample(n=preview_sample_size)}")
    
    arrow_table = df.to_arrow()

    with duckdb.connect(DEV_DB) as db:
        db.register('arrow_table', arrow_table)
        db.execute(f'CREATE TABLE IF NOT EXISTS staging.raw_{filename} AS SELECT * FROM arrow_table')

        stg_raw_table_sample = db.execute(f'SELECT * FROM staging.raw_{filename} LIMIT {preview_sample_size}').fetchdf()

        print(f"Done loading. \nSample:\n{stg_raw_table_sample}")
    

In [19]:
for i, csv_file in enumerate(CSV_FILENAMES):
    load_csv(filename=CSV_FILENAMES[i], preview_sample_size=3)

***
Loading customer data... 
Sample:
shape: (3, 5)
┌─────────────┬───────────────┬───────────────────┬────────────────────────────┬───────────────────┐
│ Customer ID ┆ Name          ┆ Customer Group ID ┆ Email                      ┆ Phone Number      │
│ ---         ┆ ---           ┆ ---               ┆ ---                        ┆ ---               │
│ i64         ┆ str           ┆ i64               ┆ str                        ┆ str               │
╞═════════════╪═══════════════╪═══════════════════╪════════════════════════════╪═══════════════════╡
│ 20          ┆ Rachel Walker ┆ 10                ┆ rachel.walker@company.net  ┆ +1 (555) 012-4567 │
│ 4           ┆ Bob Brown     ┆ 4                 ┆ bob.brown@outlook.com      ┆ +61-2-9876-5432   │
│ 14          ┆ Larry Harris  ┆ 4                 ┆ larry.harris@domain.net    ┆ +27 11 123 4567   │
└─────────────┴───────────────┴───────────────────┴────────────────────────────┴───────────────────┘
Done loading. 
Sample:
   Customer ID  


Raw data is loaded into the staging area. Now it's time to implement the **physical model** and perform the data transformations to enable business analytics. 
***

## Transformation

For transformation I am using [dbt](https://www.getdbt.com/), the most popular framework for physical data modeling and data transformations.

In [20]:
!cd air_boltic_analytics && dbt ls --resource-type=model

01:31:35  Running with dbt=1.9.2
01:31:36  Registered adapter: duckdb=1.9.2
01:31:36  Found 16 models, 12 data tests, 6 sources, 426 macros
air_boltic_analytics.marts.dim_aeroplane
air_boltic_analytics.marts.dim_customer
air_boltic_analytics.marts.dim_trip
air_boltic_analytics.marts.fct_order
air_boltic_analytics.intermediate.int_aeroplane_models
air_boltic_analytics.intermediate.int_aeroplanes
air_boltic_analytics.intermediate.int_customer_groups
air_boltic_analytics.intermediate.int_customers
air_boltic_analytics.intermediate.int_orders
air_boltic_analytics.intermediate.int_trips
air_boltic_analytics.sources.stg_aeroplane_models
air_boltic_analytics.sources.stg_aeroplanes
air_boltic_analytics.sources.stg_customer_groups
air_boltic_analytics.sources.stg_customers
air_boltic_analytics.sources.stg_orders
air_boltic_analytics.sources.stg_trips


Transformation layers:

- "Bronze" (`sources`): `stg_*` models, reflect raw data tables as is
- "Silver" (`intermediate`): `int_*` models, basic transformations like naming conventions and type casts
- "Gold" (`marts`): `dim_*` and `fct_*` models, star schema

In [21]:
!cd air_boltic_analytics && dbt run

01:31:44  Running with dbt=1.9.2
01:31:44  Registered adapter: duckdb=1.9.2
01:31:44  Found 16 models, 12 data tests, 6 sources, 426 macros
01:31:44  
01:31:44  Concurrency: 1 threads (target='dev')
01:31:44  
01:31:44  1 of 16 START sql view model main.stg_aeroplane_models ......................... [RUN]
01:31:44  1 of 16 OK created sql view model main.stg_aeroplane_models .................... [OK in 0.04s]
01:31:44  2 of 16 START sql view model main.stg_aeroplanes ............................... [RUN]
01:31:44  2 of 16 OK created sql view model main.stg_aeroplanes .......................... [OK in 0.01s]
01:31:44  3 of 16 START sql view model main.stg_customer_groups .......................... [RUN]
01:31:44  3 of 16 OK created sql view model main.stg_customer_groups ..................... [OK in 0.01s]
01:31:44  4 of 16 START sql view model main.stg_customers ................................ [RUN]
01:31:44  4 of 16 OK created sql view model main.stg_customers ........................

Possible next steps:
- During EDA, we identified missing values and inconsistent formats in the data - makes sense to find the reason and take actions. For example, there could be issues in the source systems or in the ingestion pipelines. Once we are clear about it, we can add data tests.
- Macros are a powerful mechanism in dbt, we can automate many things with them.
- Model materialization to be updated to `incremental` for models having timestamps and unique key fields.
- Use snapshots to deal with slowly changing dimensions (SCD2).
- ...

***

## CI/CD

### "Ideal" solution
The goal is to have higher quality (= more issues caught) at lower costs (running most checks before deployment).

An ideal / highly mature solution requires the use of all modern mechanisms to ensure quality, specifically:
- Data contracts to deal with boundary systems
- Unit tests to run on samples before deployment
- Data tests to run after deployment (it's a good idea to minimize these to save on costs)

The process could look like this:

```mermaid
flowchart TB

    A1(Test code) --> A[Code commit]
    B1(Test data) --> B[New data increment]

    A --> C[Run automated tests]
    B --> C

    C --> D[Code tests]
    C --> E[Data contracts]
    
    D --> F[Unit Tests on transformation]
    F --> H{Fail?}
    H -- Yes --> I[Fix issues]
    I --> C
    H -- No --> Q[Production deployment]

    %% Data tests path
    E --> K[Test data]
    E --> L[Test schema]
    K --> M{Fail?}
    L --> N{Fail?}
    M -- Yes --> O[Notification / alert]
    N -- Yes --> O
    M -- No --> Q
    N -- No --> Q
```

### "Simple" solution
The solution could be simplified as follows:

- Use simple PR review process
- Rely on dbt framework features (data contracts, unit tests, data tests)
- Use lightweight CI/CD options like GitHub Actions
- Adopt style guides and code standards

## Evolution

### Architecture
![analytics_architecture](../docs/analytics_architecture.png)

### Further considerations

From the solution architecture and engineering viewpoint, the following points can be taken into consideration:
- Open Table Formats, e.g. Apache Iceberg
- Modern open-source ELT tools, e.g. dlt
- Advanced data modeling approaches, e.g. ensemble modeling

***

If you have any questions or suggestions, please contact the author.